**현재 세션(1개)의 첫 클릭 로그**를 대상으로 MLP, Gaussian Naive Bayes, Decision Tree, XGBoost, Logistic Regression, Linear SVM을 사용해서 구매 예측

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
import itertools
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [3]:
온라인 = pd.read_csv('온라인_전처리_final_32columns.csv')

# 각 clnt_id별 session이 바뀌는 지점 index 저장
idx1 = 온라인.unique_id.drop_duplicates().index.tolist()
idx2 = idx1[1:] + [len(온라인)]

In [8]:
온라인.columns[3:-2]

Index(['hit_seq', 'hit_pss_tm', 'time_length', 'holiday_diff', 'keyword',
       'action_type_0', 'action_type_1', 'action_type_2', 'action_type_3',
       'action_type_4', 'biz_unit_A01', 'biz_unit_A02', 'biz_unit_A03',
       'sess_dt_201907', 'sess_dt_201908', 'sess_dt_201909', 'trfc_src_DIRECT',
       'trfc_src_PORTAL_1', 'trfc_src_PORTAL_2', 'trfc_src_PORTAL_3',
       'trfc_src_PUSH', 'trfc_src_WEBSITE', 'trfc_src_unknown',
       'dvc_ctg_nm_PC', 'dvc_ctg_nm_mobile_app', 'dvc_ctg_nm_mobile_web',
       'dvc_ctg_nm_unknown'],
      dtype='object')

In [9]:
len(온라인.columns[3:-2])

27

In [13]:
온라인.head()

,clnt_id,sess_id,trans_id,hit_seq,hit_pss_tm,time_length,holiday_diff,keyword,action_type_0,action_type_1,...,trfc_src_PORTAL_3,trfc_src_PUSH,trfc_src_WEBSITE,trfc_src_unknown,dvc_ctg_nm_PC,dvc_ctg_nm_mobile_app,dvc_ctg_nm_mobile_web,dvc_ctg_nm_unknown,unique_id,buy
0,1,1,NaN,1,12.0,11.0,0,1,1,0,...,0,0,0,1,0,1,0,0,1_1,0
1,1,1,NaN,2,23.0,14.0,0,2,1,0,...,0,0,0,1,0,1,0,0,1_1,0
2,1,1,NaN,3,37.0,14.0,0,3,1,0,...,0,0,0,1,0,1,0,0,1_1,0
3,1,2,NaN,1,42.0,15.0,1,1,1,0,...,0,0,0,1,0,1,0,0,1_2,0
4,1,2,NaN,2,57.0,14.0,1,2,1,0,...,0,0,0,1,0,1,0,0,1_2,0


# <br>
# 종속변수 생성
다음 세션의 구매 여부

In [10]:
# 다음 세션의 구매를 예측하기 위한 종속변수 생성
구매여부 = 온라인[['clnt_id', 'sess_id', 'buy']].groupby(['clnt_id', 'sess_id']).sum()
구매여부.buy = 구매여부.buy.apply(lambda x:0 if x == 0 else 1)
구매여부 = 구매여부.sort_index()
구매여부 = 구매여부.reset_index()
# 각 clnt_id별로 shift로 1행씩 올림
구매여부.buy = 구매여부.buy.shift(-1)
g = 구매여부.groupby('clnt_id')
# 1행씩 올리면 각 clnt_id별로 마지막 값은 다음 id의 값, 따라서 마지막 행 제거
구매여부.drop(g.tail(1).index, axis=0, inplace = True)

In [11]:
구매여부['unique_id'] = list(map(lambda x,y: str(x)+'_'+str(y), 구매여부.clnt_id, 구매여부.sess_id))

In [12]:
구매여부

,clnt_id,sess_id,buy,unique_id
0,1,1,0.0,1_1
2,2,1,0.0,2_1
3,2,2,0.0,2_2
4,2,3,0.0,2_3
5,2,4,0.0,2_4
...,...,...,...,...
367141,72427,3,0.0,72427_3
367143,72428,1,0.0,72428_1
367144,72428,2,0.0,72428_2
367145,72428,3,0.0,72428_3


In [14]:
온라인.drop(['clnt_id', 'sess_id', 'trans_id', 'buy'], axis=1, inplace=True)
온라인 = 온라인.merge(구매여부, left_on='unique_id', right_on='unique_id')
온라인.sort_values(by=['clnt_id','sess_id'], inplace=True)
온라인.reset_index(drop=True, inplace=True)

In [15]:
온라인.head()

,hit_seq,hit_pss_tm,time_length,holiday_diff,keyword,action_type_0,action_type_1,action_type_2,action_type_3,action_type_4,...,trfc_src_WEBSITE,trfc_src_unknown,dvc_ctg_nm_PC,dvc_ctg_nm_mobile_app,dvc_ctg_nm_mobile_web,dvc_ctg_nm_unknown,unique_id,clnt_id,sess_id,buy
0,1,12.0,11.0,0,1,1,0,0,0,0,...,0,1,0,1,0,0,1_1,1,1,0.0
1,2,23.0,14.0,0,2,1,0,0,0,0,...,0,1,0,1,0,0,1_1,1,1,0.0
2,3,37.0,14.0,0,3,1,0,0,0,0,...,0,1,0,1,0,0,1_1,1,1,0.0
3,1,40.0,8.0,1,1,1,0,0,0,0,...,0,0,0,0,0,1,2_1,2,1,0.0
4,2,48.0,1.0,1,1,0,1,0,0,0,...,0,0,0,0,0,1,2_1,2,1,0.0


# <br>
# 현재 세션(1개)의 첫번째 클릭 로그 사용

In [16]:
from keras.preprocessing import sequence
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.model_selection import train_test_split

In [19]:
온라인_first_hit = 온라인[온라인.hit_seq==1].copy()
온라인_first_hit.reset_index(drop=True, inplace=True)

In [20]:
온라인_first_hit.head()

,hit_seq,hit_pss_tm,time_length,holiday_diff,keyword,action_type_0,action_type_1,action_type_2,action_type_3,action_type_4,...,trfc_src_WEBSITE,trfc_src_unknown,dvc_ctg_nm_PC,dvc_ctg_nm_mobile_app,dvc_ctg_nm_mobile_web,dvc_ctg_nm_unknown,unique_id,clnt_id,sess_id,buy
0,1,12.0,11.0,0,1,1,0,0,0,0,...,0,1,0,1,0,0,1_1,1,1,0.0
1,1,40.0,8.0,1,1,1,0,0,0,0,...,0,0,0,0,0,1,2_1,2,1,0.0
2,1,14.0,10.0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,2_2,2,2,0.0
3,1,61.0,7.0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,2_3,2,3,0.0
4,1,5.0,870.0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,2_4,2,4,0.0


In [42]:
features = 온라인_first_hit.columns[:-4]

In [43]:
len(features)

27

In [21]:
온라인_x = 온라인_first_hit.iloc[:, :-4].values

In [22]:
온라인_x

array([[  1.,  12.,  11., ...,   1.,   0.,   0.],
       [  1.,  40.,   8., ...,   0.,   0.,   1.],
       [  1.,  14.,  10., ...,   0.,   0.,   1.],
       ...,
       [  1.,  21.,  21., ...,   1.,   0.,   0.],
       [  1.,  70., 556., ...,   1.,   0.,   0.],
       [  1., 244., 244., ...,   1.,   0.,   0.]])

In [24]:
# session 당 구매 여부
온라인_y = 온라인_first_hit.buy.values

In [25]:
온라인_x.shape, 온라인_y.shape

((294750, 27), (294750,))

In [26]:
def make_padding_and_oversample(X, Y):
    smote = SMOTE(random_state=0)
    X_resampled, Y_resampled = smote.fit_resample(X, Y)
    return X_resampled, Y_resampled

In [27]:
X_resampled, Y_resampled = make_padding_and_oversample(온라인_x, 온라인_y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [28]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, Y_resampled, test_size=0.3, random_state=42)
print(X_train.shape)
print(X_test.shape)

(361313, 27)
(154849, 27)


# <br>
# 모델링

In [32]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
import xgboost as xgb

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import RMSprop
from keras.layers import Dropout

import pickle
from joblib import dump, load

In [33]:
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [34]:
scores = dict()

### Gaussian Naive Bayes

In [35]:
clf = GaussianNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))
print('F1 score: %.2f' % f1_score(y_test, y_pred))
print('Precision: %.2f' % precision_score(y_test, y_pred))
print('Recall: %.2f' % recall_score(y_test, y_pred))

Accuracy: 0.60
F1 score: 0.68
Precision: 0.56
Recall: 0.84


In [36]:
scores['Gaussian Naive Bayes'] = [accuracy_score(y_test, y_pred), f1_score(y_test, y_pred),
                                  precision_score(y_test, y_pred), recall_score(y_test, y_pred)]

In [37]:
# dump(clf, '1-3-Gaussian.joblib')

### Decision Tree

In [38]:
clf2 = DecisionTreeClassifier(random_state=0).fit(X_train, y_train)
y_pred = clf2.predict(X_test)

print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))
print('F1 score: %.2f' % f1_score(y_test, y_pred))
print('Precision: %.2f' % precision_score(y_test, y_pred))
print('Recall: %.2f' % recall_score(y_test, y_pred))

Accuracy: 0.85
F1 score: 0.84
Precision: 0.87
Recall: 0.82


In [39]:
scores['Decision Tree'] = [accuracy_score(y_test, y_pred), f1_score(y_test, y_pred),
                           precision_score(y_test, y_pred), recall_score(y_test, y_pred)]

In [44]:
feature_check = {}
for name, importance in zip(features, clf2.feature_importances_):
    feature_check[name] = importance

In [45]:
pd.DataFrame.from_dict(feature_check, orient='index', columns=['feature_importance']).sort_values(by='feature_importance', ascending=False)

,feature_importance
hit_pss_tm,0.199237
time_length,0.185460
action_type_2,0.133322
holiday_diff,0.132183
trfc_src_DIRECT,0.058342
sess_dt_201907,0.046214
sess_dt_201909,0.040396
dvc_ctg_nm_mobile_app,0.030310
dvc_ctg_nm_mobile_web,0.027066
sess_dt_201908,0.025929


In [46]:
# dump(clf2, '1-3-DecisionTree.joblib')

### XGBOOST

In [41]:
clf3 = xgb.XGBClassifier(learning_rate = 0.05, n_estimators=300, max_depth=3, verbosity=2, random_state=0).fit(X_train, y_train)
y_pred = clf3.predict(X_test)

print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))
print('F1 score: %.2f' % f1_score(y_test, y_pred))
print('Precision: %.2f' % precision_score(y_test, y_pred))
print('Recall: %.2f' % recall_score(y_test, y_pred))

[16:01:36] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[16:01:37] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[16:01:37] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[16:01:37] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[16:01:38] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[16:01:38] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[16:

In [47]:
scores['XGboost'] = [accuracy_score(y_test, y_pred), f1_score(y_test, y_pred), precision_score(y_test, y_pred), recall_score(y_test, y_pred)]

In [48]:
feature_check = {}
for name, importance in zip(features, clf3.feature_importances_):
    feature_check[name] = importance

In [49]:
pd.DataFrame.from_dict(feature_check, orient='index', columns=['feature_importance']).sort_values(by='feature_importance', ascending=False)

,feature_importance
sess_dt_201908,0.152541
dvc_ctg_nm_unknown,0.103162
holiday_diff,0.101014
sess_dt_201907,0.097154
dvc_ctg_nm_mobile_app,0.088676
sess_dt_201909,0.081891
trfc_src_DIRECT,0.077419
action_type_2,0.058150
action_type_3,0.047895
keyword,0.036570


In [50]:
# dump(clf3, '1-3-xgb.joblib')

### Logistic Regression

In [52]:
clf4 = LogisticRegression(max_iter=1000, random_state=0).fit(X_train, y_train)
y_pred = clf4.predict(X_test)

print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))
print('F1 score: %.2f' % f1_score(y_test, y_pred))
print('Precision: %.2f' % precision_score(y_test, y_pred))
print('Recall: %.2f' % recall_score(y_test, y_pred))

Accuracy: 0.60
F1 score: 0.62
Precision: 0.59
Recall: 0.66


In [53]:
scores['Logistic Regression'] = [accuracy_score(y_test, y_pred), f1_score(y_test, y_pred), precision_score(y_test, y_pred), recall_score(y_test, y_pred)]

In [54]:
# dump(clf4, '1-3-LogisticRegression.joblib')

### Linear SVM

In [55]:
clf5 = LinearSVC(random_state=0).fit(X_train, y_train)
y_pred = clf5.predict(X_test)

print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))
print('F1 score: %.2f' % f1_score(y_test, y_pred))
print('Precision: %.2f' % precision_score(y_test, y_pred))
print('Recall: %.2f' % recall_score(y_test, y_pred))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Accuracy: 0.52
F1 score: 0.20
Precision: 0.61
Recall: 0.12


In [56]:
scores['Linear SVM'] = [accuracy_score(y_test, y_pred), f1_score(y_test, y_pred), precision_score(y_test, y_pred), recall_score(y_test, y_pred)]

In [57]:
# dump(clf4, '1-3-LinearSVM.joblib')

### MLP(DNN)

In [58]:
def models(train):
    model = Sequential()
    model.add(Dense(32, activation='relu', input_shape=(train.shape[1],)))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=RMSprop(lr= 0.001, rho = 0.9), metrics=['acc', f1_m, precision_m, recall_m])
    return model

In [59]:
model = models(X_train)
model.summary()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32)                896       
_________________________________________________________________
dense_2 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 1,441
Trainable params: 1,441
Non-trainable params: 0
_________________________________________________________________


In [60]:
history = model.fit(X_train, y_train, epochs=25, batch_size=1000, validation_data=(X_test, y_test), verbose=2, shuffle=True)


Train on 361313 samples, validate on 154849 samples
Epoch 1/25
 - 2s - loss: 1.1022 - acc: 0.5562 - f1_m: 0.4933 - precision_m: 0.5735 - recall_m: 0.5203 - val_loss: 1.3665 - val_acc: 0.5122 - val_f1_m: 0.1276 - val_precision_m: 0.6066 - val_recall_m: 0.0714
Epoch 2/25
 - 1s - loss: 0.9183 - acc: 0.5692 - f1_m: 0.5271 - precision_m: 0.5817 - recall_m: 0.5642 - val_loss: 0.9894 - val_acc: 0.5886 - val_f1_m: 0.6780 - val_precision_m: 0.5571 - val_recall_m: 0.8666
Epoch 3/25
 - 1s - loss: 0.8725 - acc: 0.5748 - f1_m: 0.5444 - precision_m: 0.5829 - recall_m: 0.5854 - val_loss: 0.7448 - val_acc: 0.5835 - val_f1_m: 0.5377 - val_precision_m: 0.6045 - val_recall_m: 0.4846
Epoch 4/25
 - 1s - loss: 0.8548 - acc: 0.5759 - f1_m: 0.5523 - precision_m: 0.5826 - recall_m: 0.5940 - val_loss: 0.7644 - val_acc: 0.6030 - val_f1_m: 0.6500 - val_precision_m: 0.5814 - val_recall_m: 0.7375
Epoch 5/25
 - 1s - loss: 0.8354 - acc: 0.5786 - f1_m: 0.5580 - precision_m: 0.5846 - recall_m: 0.5987 - val_loss: 1.134

In [61]:
loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, verbose=1)

154849/154849 [==============================] - 6s 41us/step


In [62]:
scores['DNN'] = [accuracy, f1_score, precision, recall]

In [63]:
print(loss)
print(accuracy)
print(f1_score)
print(precision)
print(recall)

0.7718676456676111
0.6154576539993286
0.6805914640426636
0.5796281695365906
0.8408475518226624


In [64]:
# model.save('1-3-DNN.h5')

In [65]:
pd.DataFrame.from_dict(scores, orient='index', columns=['Accuracy', 'F1-Score', 'Precision', 'Recall'])

,Accuracy,F1-Score,Precision,Recall
Gaussian Naive Bayes,0.595800,0.676377,0.564204,0.844223
Decision Tree,0.847787,0.843684,0.867668,0.820990
XGboost,0.832133,0.802330,0.976456,0.680908
Logistic Regression,0.599203,0.624095,0.587947,0.664980
Linear SVM,0.520836,0.199279,0.607914,0.119172
DNN,0.615458,0.680591,0.579628,0.840848
